In [1]:
import json
import pandas as pd
import source.train_runner_production.load_dataset as load_dataset
import cv2
import numpy as np
import matplotlib.pyplot as plt


import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

import torch
from torch.utils.data import Dataset, DataLoader

annotation_coco_path = "/root/dataset/anime_face_landmark_20230912/annotations/person_keypoints_Train.json"
data_path = "/root/dataset/anime_face_landmark_20230912/images"
dataset_list = load_dataset.cocokeypoints_list_converter(annotation_coco_path)
training_samples, valid_samples = load_dataset.train_test_split(dataset_list,0.05)
train_numpy_dataset = load_dataset.AugmentFaceKeypointDataset(training_samples,data_path,10)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training sample instances: 302
Validation sample instances: 15
オリジナル画像


In [12]:
from augument_setting import augmentation_list

data_path = "/root/dataset/anime_face_landmark_20230912/images"
aug_data_num = 30
mix_augmentent = (3,5)
data_set_list = []
# data_num = 1
for data_num in range(training_samples[0:1].shape[0]):
    image = cv2.imread(f"{data_path}/{training_samples.iloc[data_num, 0]}")
    image_orig = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    orig_h, orig_w, _ =  image_orig.shape
    # サイズ変更
    image = cv2.resize(image_orig, (512, 512))
    
    keypoints = training_samples.iloc[data_num][1:]
    keypoints = np.array(keypoints, dtype="float32")
    keypoints = keypoints.reshape(-1, 2)
    keypoints_per = keypoints * [1 / (orig_w), 1 / (orig_h)]
    data_set_list.append([image, keypoints_per])
    
    print("オリジナル画像")
    if aug_data_num == 0:
        continue

    landmark_num = len(keypoints)
    kps = KeypointsOnImage(
        [Keypoint(x=keypoints[i][0], y=keypoints[i][1]) for i in range(0,landmark_num)],
        shape=image.shape,
    )
    
    seq = iaa.SomeOf(mix_augmentent, [
        iaa.Add((-40, 40), per_channel=0.5),
        iaa.AdditiveGaussianNoise(scale=(0, 0.2*255)),
        iaa.Cutout(nb_iterations=(10, 70),size=0.05,cval=(0, 255),fill_per_channel=0.5),
        iaa.JpegCompression(compression=(80, 99)),
        iaa.BlendAlpha([0.25, 0.75], iaa.MedianBlur(13)),

        iaa.Grayscale(alpha=(0.5, 1.0)),
        iaa.LogContrast(gain=(0.6, 1.4), per_channel=True),
        iaa.Affine(scale={"x": (0.7, 1.1), "y": (0.7, 1.1)}),
        iaa.Affine(rotate=(-45, 45)),
        
        iaa.Affine(translate_percent={"x": -0.20}, mode=ia.ALL, cval=(0, 255)),
        iaa.ShearX((-20, 20)),
        iaa.ShearY((-20, 20)),
        iaa.PiecewiseAffine(scale=(0.03, 0.03)),
        iaa.imgcorruptlike.Spatter(severity=2),
        iaa.Superpixels(p_replace=0.1, n_segments=100)
    ],random_order=True)

    for aug_count in range(aug_data_num-1):
        image_aug, kps_aug = seq(image=image, keypoints=kps)
        keypoints = []
        for i in range(len(kps.keypoints)):
            before = kps.keypoints[i]
            after = kps_aug.keypoints[i]
            keypoints.append([after.x, after.y])
        keypoints = np.array(keypoints, dtype="float32")
        keypoints_per = keypoints * [1 / (orig_w), 1 / (orig_h)]
        image_after = kps_aug.draw_on_image(image_aug, size=0)

        # データ拡張を行った画像をリストに格納する
        data_set_list.append([image_after, keypoints_per])

        # 描画
        
        
        # plt.imshow(image_after)
        # plt.scatter(keypoints[:,0], keypoints[:,1], color='r',s=2)
        # plt.show()
print("len(data_set_list)",len(data_set_list))
data_set_list

オリジナル画像
len(data_set_list) 30


In [ ]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

dataset_dir = "/root/dataset/anime_face_landmark_20230912"

for data in dataset_list[-5:-1]:
    fig, ax = plt.subplots(1)
    print(data[0])
    img = Image.open(f"{dataset_dir}/images/{data[0]}")

    x_np = np.array(data[1::2])
    y_np = np.array(data[2::2])
    plt.plot(x_np, y_np,"o",color='red',markersize=3)
    
    
    
    ax.imshow(img)
    plt.show()